In [ ]:
%load_ext autoreload
%autoreload 2

from diffcam.util import LOGPATH, load_pickle, RECONSTRUCTIONPATH, DATAPATH
from scripts.evaluation import evaluate
from diffcam.util import DATAPATH, RECONSTRUCTIONPATH, print_image_info, resize, rgb2gray

import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.image as mpimg
import cv2
import collections

# Running One Test Run

In [ ]:
# Hyperparameters to set

data = 'our_images'  # Uses our image data
n_files = 1          # None yields all of our files
algo = 'glasso'      # which algorithm to run, can check the optimization file for options
n_iter = 10          # This can either be an int or a list. A list will run the cumulative sum in iterations while outputing imgages and save logs at each int value. 
gray = False         # RGB or grayscale reconstruction
lambda_ = 1e-6       # hyperparameter lambda
delta = 5            # hyperparameter delta
downsample = 4
disp = 50
flip = False
bayer = False
bg = None
rg = None
gamma = None
save = True         # Save logs and reconstructed images to folders logs and reconstruction under data folder.
plot = True         # Display images as reconstructions occur
single_psf = False
psf_fp = rf'{str(DATAPATH)}{os.sep}psf{os.sep}psf_rgb_ours.png'  #which psf to use


evaluate(data, n_files, psf_fp, algo, lambda_, delta, n_iter, downsample, disp, flip, gray, bayer, bg, rg, gamma, save, plot, single_psf)

Note: If you want to set up multiple tests you can use the run.py file which allows you to set up the running of many tests. 

# Pulling Data from Logs after runs
After we have done our run, we want to look at the data that comes from it. This can be done useing the following code. Note: Make sure to set the save flag as True in order to save the logs and all the photos.

In [ ]:
def load_data(name, and_contain = ['p'], or_contain = ['p'],  not_contains = ['zzzz']):
    """
    Use this file to load all the pickles that you want. 
    
    ex: pickle = load_data("ridge*.pkl",['.01', '1000'],['.02','.03'],[10_])
    This will match all names that start with ridge, that contain '0.1' AND '1000', that contain '.02' OR '.03' and don't contain '10_'
    
    """
    files = []
    files.extend(glob.glob(os.path.join(LOGPATH,name),recursive=True))

    pickles = []
    for f in files:
        if all(x in f for x in and_contain):
            if any(x in f for x in or_contain):
                if not any(x in f for x in not_contains):
                    pickles.append(load_pickle(f))
        
    return pickles

In [ ]:
def get_data_from_dict(pickle,
                       iters,
                       images = ['img1_rgb', 'img3_rgb', 'img4_rgb', 'img5_rgb', 'img6_rgb', 'img7_rgb', 'img8_rgb']):
    
    """
    This will pull data out of the dictionaries and put them into lists that can be manipulated.
    
    pickle: list of pickles from load_data
    iters: which iteration to pull data from
    """
    
    lambda_ = []
    algo = []
    delta = []
    
    mse = []
    psnr = []
    ssim = []
    lpips = []
    n_iter = []
    lenseless = []
    process_time = []

    for save in pickle:
        algo.append(save['algo'])
        lambda_.append(save['lambda'])
        delta.append(save['delta'])

        fp = []
        m = []
        p = []
        s = []
        l = []
        time = []
        for im in images:
            pic = save[im]
            for i in iters:
                fp.append(pic[str(i)]['recon_fp'])
                m.append(pic[str(i)]['mse'])
                p.append(pic[str(i)]['psnr'])
                s.append(pic[str(i)]['ssim'])
                l.append(pic[str(i)]['lpips'])
                time.append(pic[str(i)]['process_time'])
        lenseless.append(fp)
        mse.append(m)
        psnr.append(p)
        ssim.append(s)
        lpips.append(l)
        process_time.append(time)
    return lambda_, algo, delta, mse, psnr, ssim, lpips, n_iter, images, lenseless, process_time

In [ ]:
def get_photo(path, color = False):
    """
    Grabs the photo from the RECONSTRUCTION folder and turns it into data.
    path: path to data
    color: if data needs a color correction from cv2
    """
    
    end = os.path.split(path)[-1]
    path = os.path.join(RECONSTRUCTIONPATH, end)
    data = mpimg.imread(path)
    if color:
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    return data

Example

In [ ]:
pickle = load_data("ridge*.pkl",['.01', '1000'],['.02','.03'],[10_])

lambda_, algo, delta, mse, psnr, ssim, lpips, n_iter, images, lenseless, process_time = get_data_from_dict(pickle)

Then you can manipulate your data and we used this to plot everything in our report. The old code is in plots_old. However because of how the logging changed over time it is outdated and a bit messy. You can look at it as a reference but the code will not compile and it will not run. We also do not provide all our data in the repo since it is very heavy but you can do your own runs and use the above to pull the information from it.